# This notebook comparing the GAFF1 modelling data with the experimental value, taking the input from clustering.ipynb, in which all the results are in sampling/

In [6]:
import mdtraj as md
import numpy as np

In [23]:
def compute_ensemble_avg_NOE(grouped_files, indices, verbose=False):

    #indices = np.loadtxt(indices, dtype=int)
    results = []
    for i,state in enumerate(grouped_files):
        distances = md.compute_distances(md.load(state), indices)*10. # convert nm to Å
        print(distances.shape)
        data = np.mean(np.array(distances), axis=0)
        results.append(data)
        #print(np.mean([d[0][1] for d in distances]))
        #print(data)
        #exit()
        #np.savetxt(f'{outdir}/NOE_{i}.txt', data)
    return results

In [24]:
import numpy as np

# Sample gmx_indices and restraint_indices
gmx_indices = [
    [12, 31],
    [12, 32],
    [13, 31],
    [13, 32],
    [40, 53],
    [40, 54],
    [41, 53],
    [41, 54],
]

d_indices = np.array(gmx_indices) - 1  # Convert to 0-based indexing

# Assuming pdb is a list of pdb file paths
pdb = ['sampling/sample_state_0_cluster2_whole.pdb']

# Compute distances
distances = compute_ensemble_avg_NOE(pdb, d_indices)
print("Distances:", distances)

# Provided restraint_indices
restraint_indices = np.array([0, 0, 0, 0, 1, 1, 1, 1])

# Initialize dictionary to hold distances for each state
state_distances = {state: [] for state in np.unique(restraint_indices)}

# Assign distances to the corresponding state
for state in state_distances:
    state_distances[state] = distances[restraint_indices == state]

# Calculate average distance for each state
avg_state_distances = {state: np.mean(dists) for state, dists in state_distances.items()}

print("Average distances for each state:", avg_state_distances)


(10, 8)
Distances: [array([2.9317842, 3.2018027, 3.0226872, 3.2559128, 2.8340402, 3.5889611,
       2.7757835, 3.699246 ], dtype=float32)]


TypeError: only integer scalar arrays can be converted to a scalar index